In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import itertools as itr
import copy
from boundaries import WallBoundary, PressureBoundary, VelocityBoundary
from generators import WallGenerators
from simulation import Simulation
import pandas as pd
import random
from openpyxl import load_workbook

In [ ]:
# new = pd.DataFrame({
#             "BestPos" : [],
#             "BestDirection" : [],
#             "BestMomentum" : [],
#             "ImageLink": [],
#             "SizeX": [],
#             "SizeY": [],
#             "NumberOfCuts": [],
#             "TypesOfCuts": [],
#             "CutPositionsX": [],
#             "CutPositionsY": [],
#             "CutSizesX": [],
#             "CutSizesY": [] })

In [ ]:
data_path = 'data/data.xlsx' 
data = pd.read_excel(data_path).iloc[:, 1:]

In [ ]:
count = len(data.index)
print(count)

In [ ]:
#count = 3

In [ ]:
new = pd.DataFrame({
            "BestX" : [],
            "BestY" : [],
            "BestTime" : [],
            "WorstTime" : [],
            "ImageLink": [],
            "SizeX": [],
            "SizeY": [],
            "NumberOfCuts": [],
            "TypesOfCuts": [],
            "CutPositionsX": [],
            "CutPositionsY": [],
            "CutSizesX":  [],
            "CutSizesY": [] })
for i in range(1000000): 
    yResolution = 32
    xResolution = 32
    ACspeed = 1

    walls = WallBoundary(yResolution, xResolution)
    walls.borderWall()
    conditions = walls.generateRoom()
    walls.generateIndex()
    walls.generateACDirections()
    walls.indexWithoutCare()

    # real = np.sum(walls.possibleACPos)
    # bad = len(walls.possibleACIndex)
    # if bad/real < 0.8:
    #     print("BAD")
    #     continue

    # bestPos = []
    # bestDirection = []
    # bestTime = 501

    total = len([i for i in walls.possibleACIndex])
    print(f"{count}")
    print(f"Total Number Of Cobinations: {total}")
    start = 1
    print(walls.possibleACIndex)

    #plt.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")

    results = []

    for index, pos in enumerate(walls.possibleACIndex):
        print(f"Simulation batch {start}/{total}")
        simTimes = []
        for directionIndex, direction in enumerate(WallBoundary.unitVect):
            #print(f"Trying direction: {direction}...")
            pathOfFlow = [i+j for i, j in zip(pos, direction)]
            if (not walls.boundary[pathOfFlow[0], pathOfFlow[1]]):
                #print("Works! Testing now...")
                velocityInlet = [VelocityBoundary(pos[0], pos[1], ACspeed, directionIndex)]
                initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
                simulation = Simulation(
                    yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=[]
                )
                simTime, stable = simulation.simulateUntilEquilibrium(equilibriumThreshold=2.5, limit = 500)
                #print("Done!")
                if not stable or simTime == 500:
                    #print("It exploded :() or didn't equalize")
                    break
                simTimes.append(simTime)
            #else:
                #print("Doesn't work! Moving on...")
            #print("\n")
        results.append(np.average(simTimes))
        start += 1
        #print("--------------")
    
    if(len(results) == 0):
        continue

    full_frame() 
    plt.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")
    imgpath = f'data/images/{count}.png'
    plt.savefig(imgpath, bbox_inches="tight", pad_inches=0)
    
    results = np.nan_to_num(results, copy=False, nan=501)

    lowest = min(results)
    most = max(results)
    ansIndex = [index for index, result in enumerate(results) if result == lowest][0]
    ansPos = walls.possibleACIndex[ansIndex]

    conditions["BestX"] = [ansPos[1]]
    conditions["BestY"] = [ansPos[0]]
    conditions["BestTime"] = [lowest]
    conditions["WorstTime"] = [most]
    conditions["ImageLink"] = [imgpath]

    print(f"position: {ansPos} Momentum: {results[ansIndex]}")
    df_dictionary = pd.DataFrame(conditions)
    new = pd.concat([new, df_dictionary], ignore_index=True)
    count += 1


In [51]:
results = np.nan_to_num(results, copy=False, nan=501)
print(results)

[  5.66666667  26.           5.          26.25        24.75
   6.          24.25         6.66666667  26.5         30.66666667
  47.          49.66666667  31.         116.          44.66666667
 501.           5.66666667  25.5         25.75        30.
  44.66666667  47.33333333   6.25        38.5         23.5
  42.33333333  24.33333333  34.5         50.          53.75
  24.          59.          53.5         53.25        36.5
  37.75        39.          25.25        25.25         6.
  23.75         6.66666667  39.5         29.66666667  38.
  27.66666667  50.5         50.33333333  49.75        49.33333333
  49.5         69.66666667  49.5         69.33333333  49.75
  69.        ]


In [52]:
data = pd.concat([data, new], ignore_index=True)
print(data.head())

   BestX  BestY  BestTime   WorstTime          ImageLink  SizeX  SizeY  \
0    2.0    4.0       6.0  146.250000  data/images/0.png   32.0   32.0   
1    1.0   30.0       6.5   86.000000  data/images/1.png   32.0   32.0   
2   12.0    1.0       5.5   92.200000  data/images/2.png   32.0   32.0   
3   29.0    5.0       6.0  129.250000  data/images/3.png   32.0   32.0   
4   30.0    1.0       6.5   91.333333  data/images/4.png   32.0   32.0   

   NumberOfCuts         TypesOfCuts           CutPositionsX  \
0           5.0     [1, 0, 1, 1, 1]       [31, 0, 0, 0, 31]   
1           1.0                 [1]                     [0]   
2           5.0     [0, 1, 0, 1, 0]      [0, 16, 0, 31, 16]   
3           6.0  [0, 1, 0, 1, 1, 1]  [31, 0, 16, 0, 31, 31]   
4           1.0                 [0]                     [0]   

           CutPositionsY           CutSizesX           CutSizesY  
0     [31, 16, 0, 31, 0]     [4, 6, 9, 5, 4]     [4, 6, 6, 9, 5]  
1                   [16]                 [

In [53]:
data.to_excel(data_path)

In [ ]:
lowest = min(results)
ansIndex = [index for index, result in enumerate(results) if result == lowest][0]
ansPos = walls.possibleACIndex[ansIndex]
print(ansIndex)
print(ansPos)
print(results[ansIndex])
print(list(enumerate(results)))

In [ ]:
print(conditions)

In [ ]:

# bestPos = []
# bestDirection = []
# bestTemp = 0

# total = len([i for i in walls.possibleACIndex])
# print(f"{count}")
# print(f"Total Number Of Cobinations: {total}")
# start = 1
# print(walls.possibleACIndex)

# #plt.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")

# for index, pos in enumerate(walls.possibleACIndex):
#     print(f"Simulation batch {start}/{total}")
#     for directionIndex, direction in enumerate(WallBoundary.unitVect):
#         #print(f"Trying direction: {direction}...")
#         pathOfFlow = [i+j for i, j in zip(pos, direction)]
#         if (not walls.boundary[pathOfFlow[0], pathOfFlow[1]]):
#             #print("Works! Testing now...")
#             velocityInlet = [VelocityBoundary(pos[0], pos[1], ACspeed, directionIndex)]
#             initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
#             simulation = Simulation(
#                 yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=[]
#             )
#             simTime, stable = simulation.simulateUntilEquilibrium(equilibriumThreshold=2.5, limit = 500)
#             results = simulation.getMomentum()
#             #print("Done!")
#             if not stable or simTime == 500:
#                 #print("It exploded :() or didn't equalize")
#                 break
#             if results > bestTemp:
#                 bestPos = pos
#                 bestDirection = directionIndex
#                 bestTemp = results
#                 print(f"It works! Momentum: {results}")
#         #else:
#             #print("Doesn't work! Moving on...")
#         #print("\n")
#     start += 1
#     #print("--------------")
    

# conditions["BestPos"] = [bestPos]
# conditions["BestDirection"] = [bestDirection]
# conditions["BestMomentum"] = [bestTemp]
# conditions["ImageLink"] = [imgpath]

# print(f"position: {bestPos} direction: {bestDirection} Momentum: {bestTemp}")
# df_dictionary = pd.DataFrame(conditions)
# data = pd.concat([data, df_dictionary], ignore_index=True)
# count += 1


In [ ]:
# yResolution = random.randint(20,50)
# xResolution = random.randint(20,50)
# ACspeed = 1

# walls = WallBoundary(yResolution, xResolution)
# walls.borderWall()
# conditions = walls.generateRoom()
# walls.generateIndex()
# walls.generateACDirections()
# walls.indexPossibleACPos()

# plt.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")
# imgpath = f'data/images/{count}.png'
# plt.savefig(imgpath)

In [ ]:
# velocityInlet = [VelocityBoundary(bestPos[0], bestPos[1], 1, bestDirection)]
# initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
# simulation = Simulation(
#     yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=[]
# )


In [ ]:
# simulation.simulate(step = 10)
# densityPlot = copy.copy(simulation.density)
# densityPlot[walls.invertedBoundary] -= np.min(densityPlot[walls.invertedBoundary])
# print(np.average(simulation.fluid))
# print(simulation.getMomentum())

# fig, ax = plt.subplots()
# heatmap = ax.imshow(densityPlot, cmap = "rainbow", interpolation = "nearest")
# quivermap = ax.quiver(simulation.xIndex, simulation.yIndex, simulation.momentumX, -simulation.momentumY)
# fig.colorbar(heatmap)
# # print(simulation.fluid)
# plt.show()

In [ ]:
# velocityInlet = [VelocityBoundary(bestPos[0], bestPos[1], 1, )]
# initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
# simulation = Simulation(
#     yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=[]
# )


In [ ]:
# simulation.simulate(step = 10)
# densityPlot = copy.copy(simulation.density)
# densityPlot[walls.invertedBoundary] -= np.min(densityPlot[walls.invertedBoundary])
# print(np.average(simulation.fluid))
# print(simulation.getMomentum())

# fig, ax = plt.subplots()
# heatmap = ax.imshow(densityPlot, cmap = "rainbow", interpolation = "nearest")
# quivermap = ax.quiver(simulation.xIndex, simulation.yIndex, simulation.momentumX, -simulation.momentumY)
# fig.colorbar(heatmap)
# # print(simulation.fluid)
# plt.show()